
# 03 临界慢化

目标：验证 r 靠近 rc 左侧时 Lag-1 自相关上升，作为早期预警信号。


In [ ]:

import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", palette="deep")

ROOT = Path.cwd()
if not (ROOT / 'src').exists() and (ROOT.parent / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src import calculate_chi, calculate_rc, get_gl_params
from src.sde_solver import SDEConfig, run_sde_simulation
from src.utils import calculate_autocorrelation

phi, theta, k_avg = 0.6, 0.4, 50
n_m, n_w = 10, 5
output_dir = ROOT / 'outputs'
output_dir.mkdir(exist_ok=True)
seed = 0


In [ ]:

chi = calculate_chi(phi=phi, theta=theta, k_avg=k_avg)
rc = float(calculate_rc(n_m=n_m, n_w=n_w, chi=chi))
print(f"chi={chi:.3f}, rc={rc:.3f}")


In [ ]:

r_vals = np.linspace(rc - 0.12, rc - 0.01, 6)
lag1_list = []
for r in r_vals:
    alpha, u = get_gl_params(r=r, rc=rc)
    cfg = SDEConfig(alpha=float(alpha), u=u, sigma=0.15, dt=1e-2, steps=6000, n_trajectories=4, seed=seed)
    t, traj = run_sde_simulation(cfg, q0=0.0, record_interval=20)
    acs = [calculate_autocorrelation(traj[:, i], lag=1) for i in range(traj.shape[1])]
    lag1_list.append(float(np.mean(acs)))

csd_df = pd.DataFrame({'r': r_vals, 'lag1_ac': lag1_list})
csd_df.to_csv(output_dir / 'critical_slowing_down.csv', index=False)


In [ ]:

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(r_vals, lag1_list, marker='o', color='tab:purple', label='Lag-1 autocorrelation')
ax.axvline(rc, color='gray', linestyle='--', label='r_c')
ax.set_xlabel('Removal ratio r (stable side)')
ax.set_ylabel('Lag-1 autocorrelation')
ax.set_title('Critical slowing down')
ax.legend()
fig.tight_layout()
fig.savefig(output_dir / 'critical_slowing_down.png', dpi=200)
fig



**结论**：r → r_c 左侧时自相关上升，恢复力减弱、记忆效应增强，符合临界慢化预警特征。
